In [1]:
import pandas as pd
import numpy as np

%matplotlib inline
from matplotlib import pyplot as plt

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6
plt.figure(figsize=(10,10))

from statsmodels.tools import eval_measures
import statsmodels.formula.api as smf
import statsmodels.api as sm

<Figure size 720x720 with 0 Axes>

In [2]:
def preprocess_data(data_path, labels_path=None):
    # load data and set index to city, year, weekofyear
    df = pd.read_csv(data_path, index_col=[0, 1, 2])
    
    # select features we want
    features = ['reanalysis_specific_humidity_g_per_kg', 
                 'reanalysis_dew_point_temp_k', 
                 'station_avg_temp_c', 
                 'station_min_temp_c']
    df = df[features]
    
    # fill missing values
    df.fillna(method='ffill', inplace=True)

    # add labels to dataframe
    if labels_path:
        labels = pd.read_csv(labels_path, index_col=[0, 1, 2])
        df = df.join(labels)
    
    # separate san juan and iquitos
    sj = df.loc['sj']
    iq = df.loc['iq']
    
    return sj, iq

In [3]:
sj_train, iq_train = preprocess_data('./dengue_features_train.csv', labels_path="./dengue_labels_train.csv")

In [4]:
print(sj_train.shape)
print(iq_train.shape)

(936, 5)
(520, 5)


In [5]:
sj_subtrain = sj_train.head(500)
sj_subtest = sj_train.tail(sj_train.shape[0] - 500)

iq_subtrain = iq_train.head(300)
iq_subtest = iq_train.tail(iq_train.shape[0] - 300)

generalized liner model


In [6]:
def get_best_model(train, test):
    # Step 1: specify the form of the model
    model_formula = "total_cases ~ 1 + reanalysis_specific_humidity_g_per_kg + reanalysis_dew_point_temp_k + " \
                    "station_min_temp_c + station_avg_temp_c"
    
    grid = 10 ** np.arange(-8, -3, dtype=np.float64)
                    
    best_alpha = []
    best_score = 1000
        
    # Step 2: Find the best hyper parameter, alpha
    for alpha in grid:
        
        model = smf.glm(formula=model_formula, data=train, 
                        family=sm.families.NegativeBinomial(alpha=alpha))

        results = model.fit()
        predictions = results.predict(test).astype(int)
        score = eval_measures.meanabs(predictions, test.total_cases)

        if score < best_score:
            best_alpha = alpha
            best_score = score

    print('best alpha = ', best_alpha)
    print('best score = ', best_score)
            
    # Step 3: refit on entire dataset
    full_dataset = pd.concat([train, test])
    model = smf.glm(formula=model_formula,
                    data=full_dataset,
                    family=sm.families.NegativeBinomial(alpha=best_alpha))

    fitted_model = model.fit()
    return fitted_model